In [1]:
import re

#### 1-Eliminate implication

In [106]:
#done
def E_I(exp):

    pat = r'(\b\w+\b)\s*->\s*(\b\w+\b)'
    return re.sub(pat, r' ~\1 | \2 ', exp)

logical_expression = "(P -> Q)"
eliminated_expression = E_I(logical_expression)
print("Original expression:", logical_expression)
print("Expression with implications eliminated:", eliminated_expression)


Original expression: (P -> Q)
Expression with implications eliminated: ( ~P | Q )


#### 2-Move negation inward (Demorgan Law)

In [97]:
def MNI(expression):
    # Loop until all negations (~) have been moved inward
    while True:
        # Find the start index of the next negation (~)
        negation_start = expression.find('~(')
        # If no negation is found, exit the loop
        if negation_start == -1:
            break
        
        # Find the corresponding closing parenthesis for the negation
        counter = 1
        negation_end = negation_start + 2
        for i in range(negation_start + 2, len(expression)):
            if expression[i] == '(':
                open_paren_count += 1
            elif expression[i] == ')':
                open_paren_count -= 1
                if open_paren_count == 0:
                    negation_end = i
                    break
        
        # Extract the inner formula enclosed by the negation
        inner_formula = expression[negation_start + 2:negation_end]
        
        # Split the inner formula by the connective symbols (| or &)
        subExp = inner_formula.split('|') if '|' in inner_formula else inner_formula.split('&')
        
        # Apply negation to each subformula and reconstruct the inner formula
        negated_subformulas = ['~' + subformula if subformula[0] != '~' else subformula[1:] for subformula in subExp]
        inner_formula = '(' + '|'.join(negated_subformulas) + ')' if '| ' in inner_formula else '(' + '& '.join(negated_subformulas) + ')'
        
        # Replace the negated expression with the transformed inner formula
        expression = expression[:negation_start] + inner_formula + expression[negation_end + 1:]
    
    # Return the expression with all negations moved inward
    return expression

# Example usage:
formula = "∀x(~(P(x) & Q(x)) | ~R(x))"
moved_formula = MNI(formula)
print("Original formula:", formula)
print("Formula with negation moved inward:", moved_formula)


Original formula: ∀x(~(P(x) & Q(x)) | ~R(x))
Formula with negation moved inward: ∀x((~P(x) & ~ Q(x)) | ~R(x))


####  3-Remove double-not.

In [67]:
#done
def remove_doube(exp):
    result = []
    i = 0
    while i < len(exp):
        if exp[i:i+2] == '~~':
            # Skip over the double negation
            i += 2
        else:
            result.append(exp[i])
            i += 1
    return ''.join(result)

logical_expression = "~~(P & Q) "
expression_without_double_negation = remove_doube(logical_expression)
print("Original expression:", logical_expression)
print("Expression without double negation:", expression_without_double_negation)

Original expression: ~~(P & Q) 
Expression without double negation: (P & Q) 


####  4-Standardize variable scope

In [173]:
def SVS(exp):
    l = []
    for expression in exp:
        new_expr = expression
        used_variables = set()
        rep_map = {}
        def new_variable(old_var):
            if old_var in rep_map:
                return rep_map[old_var]
            new_var = chr(ord('a') + len(used_variables))
            while new_var in used_variables:
                new_var = chr(ord(new_var) + 1)
            used_variables.add(new_var)
            rep_map[old_var] = new_var
            return new_var

        i = 0
        while i < len(new_expr):
            if new_expr[i] in "∀∃":
                var = new_expr[i + 1]
                if var.islower(): # Confirm variable
                    new_var = new_variable(var)
                    new_expr = new_expr[:i + 1] + new_expr[i + 1:].replace(var, new_var)
                    i += len(new_var) - 1
            i += 1
        l.append(new_expr)

    return l

# Test case
# Test case 1
expressions_1 = ['∃x(P(x) & Q(x)) '  ' ∀x(R(x) | S(x))']
print("Original Expressions 1:", expressions_1)
print("SVS Result 1:", SVS(expressions_1))


# Test case 3
expressions_2 = ['∃x∀y(P(x, y) & Q(x, y))' '∀z∃w(R(z, w) | S(z, w))']
print("\nOriginal Expressions 3:", expressions_2)
print("SVS Result 3:", SVS(expressions_2))


Original Expressions 1: ['∃x(P(x) & Q(x))  ∀x(R(x) | S(x))']
SVS Result 1: ['∃a(P(a) & Q(a))  ∀b(R(b) | S(b))']

Original Expressions 3: ['∃x∀y(P(x, y) & Q(x, y))∀z∃w(R(z, w) | S(z, w))']
SVS Result 3: ['∃a∀b(P(a, b) & Q(a, b))∀c∃d(R(c, d) | S(c, d))']


#### 5- The prenex form (obtained by moving all quantifiers to the left of the
formula.)

In [66]:
#done
def penex_form(exp):
    quanti_part = ""
    parts = ""
    i = 0
    while i < len(exp):
        if exp[i] == '∀' or exp[i] == '∃':
            quanti_part += exp[i] + exp[i+1] + ' '
            i += 1  # Skip the next character which is the variable
        else:
            parts += exp[i]
        i += 1
    return quanti_part.strip() + " " + parts.strip()

# Test case
expression = '∃x(P(x) ∧ Q(x)) ∧ ∀y(R(y) ∨ S(y))'
print("Original Expression:", expression)
print("Penex Form:", penex_form(expression))


Original Expression: ∃x(P(x) ∧ Q(x)) ∧ ∀y(R(y) ∨ S(y))
Penex Form: ∃x ∀y (P(x) ∧ Q(x)) ∧ (R(y) ∨ S(y))


#### 6-Skolemization for existential quantifiers

In [151]:
def skolemization(exp):
    parts = exp.split('∃')
    skolemized_parts = [parts[0]]
    for part in parts[1:]:
        if part.strip():
            variables, _, inner_exp = part.partition('(')
            variables += '('  # Adding back the opening parenthesis
            
            # Create Skolem function with parentheses
            skolem_function = "(f" + variables.replace('(', '').replace(')', ')')  # Add closing parenthesis
            
            # Replace variable x in the inner expression with the Skolem function
            skolemized_inner_exp = inner_exp.replace('x', skolem_function)
            
            
            # Append skolemized inner expression to the list of skolemized parts
            skolemized_parts.append(skolemized_inner_exp)
    return ''.join(skolemized_parts)

# Example usage:
logical_expression = "∃x (P(x) & Q(x))"
skolemized_expression = skolemization(logical_expression)
print("Original expression:", logical_expression)
print("Skolemized expression:", skolemized_expression)


Original expression: ∃x (P(x) & Q(x))
Skolemized expression: P((fx ) & Q((fx ))


#### 7-Eliminate universal quantifiers.

In [71]:
# done
def E_U(expression):
    # Define a pattern to match universal quantifiers and their corresponding variables and inner expressions
    pattern = r'∀(\w+)\s*\((.*?)\)'

    # Use regex to remove all occurrences of universal quantifiers
    expression_without_universal_quantifiers = re.sub(pattern, r'(\2)', expression)

    return expression_without_universal_quantifiers

# Example usage:
logical_expression = "∀x (P(x) & Q(x))"
expression_without_universal_quantifiers = E_U(logical_expression)
print("Original expression:", logical_expression)
print("Expression without universal quantifiers:", expression_without_universal_quantifiers)


Original expression: ∀x (P(x) & Q(x))
Expression without universal quantifiers: (P(x) & Q(x))
